In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

qwen3_4b_4bit_training_path = "local/output/runs/current/events.out.tfevents.1750742840.r1-distil-qapo-rollout-only-v1-0.38984.0"

In [ ]:
ea = EventAccumulator(qwen3_4b_4bit_training_path)
ea.Reload()

print(ea.Tags()["scalars"])

In [1]:
import os
import torch
from transformers.models.auto.modeling_auto import AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from safetensors.torch import safe_open
import torch

file_paths = [
    "Qwen/Qwen3-4B-4bit/model-00001-of-00004.safetensors",
    "Qwen/Qwen3-4B-4bit/model-00002-of-00004.safetensors",
    "Qwen/Qwen3-4B-4bit/model-00003-of-00004.safetensors",
    "Qwen/Qwen3-4B-4bit/model-00004-of-00004.safetensors",
]

state_dict = model.state_dict()
state_dict_v2 = {}

for file_path in file_paths:
    with safe_open(file_path, framework="pt") as f:
        for key in f.keys():
            x = f.get_tensor(key)
            if key in state_dict and x.shape != state_dict[key].shape:
                print(key, x.shape, state_dict[key].shape)
            
            state_dict_v2[key] = x

In [9]:
[k for k in state_dict_v2 if k not in state_dict]

['model.layers.0.mlp.down_proj.weight_clip_val',
 'model.layers.0.mlp.gate_proj.weight_clip_val',
 'model.layers.0.mlp.up_proj.weight_clip_val',
 'model.layers.0.self_attn.k_proj.weight_clip_val',
 'model.layers.0.self_attn.o_proj.weight_clip_val',
 'model.layers.0.self_attn.q_proj.weight_clip_val',
 'model.layers.0.self_attn.v_proj.weight_clip_val',
 'model.layers.1.mlp.down_proj.weight_clip_val',
 'model.layers.1.mlp.gate_proj.weight_clip_val',
 'model.layers.1.mlp.up_proj.weight_clip_val',
 'model.layers.1.self_attn.k_proj.weight_clip_val',
 'model.layers.1.self_attn.o_proj.weight_clip_val',
 'model.layers.1.self_attn.q_proj.weight_clip_val',
 'model.layers.1.self_attn.v_proj.weight_clip_val',
 'model.layers.2.mlp.down_proj.weight_clip_val',
 'model.layers.2.mlp.gate_proj.weight_clip_val',
 'model.layers.2.mlp.up_proj.weight_clip_val',
 'model.layers.2.self_attn.k_proj.weight_clip_val',
 'model.layers.2.self_attn.o_proj.weight_clip_val',
 'model.layers.2.self_attn.q_proj.weight_clip

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for key in ["train/loss", "eval/loss"]:
    step = np.array([event.step for event in ea.Scalars(key)])
    value = np.array([event.value for event in ea.Scalars(key)])
    plt.plot(step, value)
    plt.grid()
    plt.xlabel("step")
    plt.ylabel(key)
    plt.show()